# Sentiment Analysis Trial #3

In [1]:
from textblob import TextBlob
import pandas as pd
import sys
import tweepy
import matplotlib.pyplot as plt
import numpy as np
import os
import seaborn as sns
import time
import re
import string

import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import SnowballStemmer
from nltk.corpus import stopwords
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from nltk.stem import WordNetLemmatizer 
from nltk.corpus import wordnet
nltk.download('averaged_perceptron_tagger')
nltk.download('vader_lexicon')
nltk.download('stopwords')
nltk.download('wordnet')

!pip install gensim
import gensim
from gensim.parsing.preprocessing import remove_stopwords 
import torch
import flair
from flair.models import TextClassifier
from flair.data import Sentence
from segtok.segmenter import split_single

from wordcloud import WordCloud, STOPWORDS
from PIL import Image
from langdetect import detect
from sklearn.feature_extraction.text import CountVectorizer

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/memme11/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/memme11/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/memme11/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/memme11/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [6]:
pd.set_option('display.max_colwidth', None)

In [7]:
df = pd.read_csv('Cleaned_Bitcoin_tweets.csv')
df

Date  \
0      2021-07-05 23:26:41   
1      2021-07-05 23:26:36   
2      2021-07-05 23:25:27   
3      2021-07-05 23:25:19   
4      2021-07-05 23:24:35   
...                    ...   
48680  2021-02-05 11:00:24   
48681  2021-02-05 11:00:24   
48682  2021-02-05 11:00:22   
48683  2021-02-05 11:00:21   
48684  2021-02-05 10:58:47   

                                                                                                                                                                                                                                                                                                                    text  \
0      #Thailand #Boutiquehotels For Sale! \n#willroadhouse #AoyRoadhouse #compassgroupinternational #ThailandBeachfrontHomes #ThailandBeachfrontHome  #retirement \n#thailandrealestate #phuket #bkk\n#bangkok #huahin #jomtienbeach #jomtien #pattaya #thailandexpats #bitcoin #cryptocurrency https://t.co/GbFTpmaesz   
1                                LUNALAND ($LLN) will burn coin supply until we reach 1 Lunar distance in coins. (400M coins) A burn rate of 99.96% from the initial pool of 1 Trillion coins reduces coin supply, #presale #blockchain #cryptotrading #investing #cryptocurrency #bitcoin #trading #altcoin #cryptonews   
2                    #LightningNetwork \n\nhttps://t.co/Q0CaYRhLWD observed:\n22,365 nodes (12,502 with active channels)\n53,035 channels\n1,682.728 BTC capacity ($57,335,747)\n\nmedian node capacity: 0.002 BTC ($74.64)\n\npast 24h:\n+72 nodes +972 channels +17.078 BTC ($581,892)\n\n#Lightning #LN #bitcoin $BTC   
3                                        Barclay's just wants to keep you safe, this is why they let you spend your life savings on the lottery, at the casino, or trading stock futures. But buying #bitcoin  on Binance is a bridge to far. \n\nPS they are a huge criminal money launderer\n\nhttps://t.co/FUb9BMnavs   
4                                                                                                                                                                                                                                Why 'Cardano is just teasing us' - https://t.co/8JowtgVKvj $ADA #altcoin #news #bitcoin   
...                                                                                                                                                                                                                                                                                                                  ...   
48680                                                                                                                                                                       5⃣   #Blockchain 50 2021 by @DelRayMan, @Forbes , @ForbesCrypto  ⬇️\n\n#cryptocurrency #bitcoin #crypto #blockchain… https://t.co/1g8ppqVtqA   
48681                                                                                                                                                                       4⃣  🎙️ Bloomberg LP #CryptoOutlook 2021 with @mikemcglone11 ⬇️\n\n#cryptocurrency #bitcoin #crypto #blockchain #btc… https://t.co/re2bksq9tW   
48682                                                                                                                                                                             2⃣   Debunking 9 #Bitcoin Myths by @Patrick_Lowry_  ⬇️  \n\n#cryptocurrency #bitcoin #crypto #blockchain #btc… https://t.co/AYJQsuJmso   
48683                                                                                                                                                                   📖  Weekend Read 📖\n\nKeen to learn about #crypto assets? Check out our weekend reading list🙌\n\n1⃣2021 CryptoMarket Out… https://t.co/jjN4zeHtXY   
48684                                                                                                                                                                       #Bitcoin brac

In [8]:
# Sentiment
tweets = df.head(500)

In [9]:
def get_wordnet_pos(word):
#Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}

    return tag_dict.get(tag, wordnet.NOUN)

def second_clean(tweet, lemmatize = 'nltk'):
    tweet = remove_stopwords(tweet) # remove stopwords with Gensim

    if (lemmatize == 'spacy'):
        # Initialize spacy 'en' model, keeping only tagger component needed for lemmatization
        nlp = spacy.load('en', disable=['parser', 'ner'])
        doc = nlp(tweet)
        tokenized = [token.lemma_ for token in doc if token.lemma_ != '-PRON-']
    elif (lemmatize == 'nltk'): 

        lemmatizer = WordNetLemmatizer()
        tokenized = [lemmatizer.lemmatize(w, get_wordnet_pos(w)) for w in nltk.word_tokenize(tweet)]
    
    # remove left over stop words with nltk
    tokenized = [token for token in tokenized if token not in stopwords.words("english")] 

    # remove non-alpha characters and keep the words of length >2 only
    tokenized = [token for token in tokenized if token.isalpha() and len(token)>2]

    return tokenized

def combine_tokens(tokenized): 
    non_tokenized = ' '.join([w for w in tokenized])
    return non_tokenized

In [10]:
# Execute function 
tweets['tokens'] = tweets['cleaned_text'].apply(lambda x: second_clean(x, 'nltk'))
tweets['final_clean'] = tweets['tokens'].apply(lambda x: combine_tokens(x))

/Users/memme11/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/memme11/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [13]:
column_order = ['Date','text', 'cleaned_text', 'tokens', 'final_clean', 'user_followers', 'user_friends', 'user_favourites', 'is_retweet', 'hashtags']
tweets = tweets[column_order]
tweets

Date  \
0    2021-07-05 23:26:41   
1    2021-07-05 23:26:36   
2    2021-07-05 23:25:27   
3    2021-07-05 23:25:19   
4    2021-07-05 23:24:35   
..                   ...   
495  2021-07-05 20:24:17   
496  2021-07-05 20:23:10   
497  2021-07-05 20:23:08   
498  2021-07-05 20:22:53   
499  2021-07-05 20:22:40   

                                                                                                                                                                                                                                                                                                                  text  \
0    #Thailand #Boutiquehotels For Sale! \n#willroadhouse #AoyRoadhouse #compassgroupinternational #ThailandBeachfrontHomes #ThailandBeachfrontHome  #retirement \n#thailandrealestate #phuket #bkk\n#bangkok #huahin #jomtienbeach #jomtien #pattaya #thailandexpats #bitcoin #cryptocurrency https://t.co/GbFTpmaesz   
1                              LUNALAND ($LLN) will burn coin supply until we reach 1 Lunar distance in coins. (400M coins) A burn rate of 99.96% from the initial pool of 1 Trillion coins reduces coin supply, #presale #blockchain #cryptotrading #investing #cryptocurrency #bitcoin #trading #altcoin #cryptonews   
2                  #LightningNetwork \n\nhttps://t.co/Q0CaYRhLWD observed:\n22,365 nodes (12,502 with active channels)\n53,035 channels\n1,682.728 BTC capacity ($57,335,747)\n\nmedian node capacity: 0.002 BTC ($74.64)\n\npast 24h:\n+72 nodes +972 channels +17.078 BTC ($581,892)\n\n#Lightning #LN #bitcoin $BTC   
3                                      Barclay's just wants to keep you safe, this is why they let you spend your life savings on the lottery, at the casino, or trading stock futures. But buying #bitcoin  on Binance is a bridge to far. \n\nPS they are a huge criminal money launderer\n\nhttps://t.co/FUb9BMnavs   
4                                                                                                                                                                                                                              Why 'Cardano is just teasing us' - https://t.co/8JowtgVKvj $ADA #altcoin #news #bitcoin   
..                                                                                                                                                                                                                                                                                                                 ...   
495                                                                                        #linkedin #twitter #giftidea #facebook #instagram #tumblr #socialmedia #pinterest #dogecoin #reddit #gift #gifts #giftideas #shop #shopping #affiliatemarketing #bitcoin Shop #gifts &amp; #baskets https://t.co/dnN7uRmi6e   
496                                                                                                                                   Argentina vs. Colombia: Copa America semifinals live stream, TV channel, how to watch online, news, odds https://t.co/ajfyDCgPlj ▶️ https://t.co/NKkDBVS66N #Bitcoin #Sportsbook   
497                                                                                                                                           Brazil vs. Peru: Copa America semifinal live stream, TV channel, how to watch online, news, odds https://t.co/P86c6Ag9tV ▶️ https://t.co/NKkDBVS66N #Bitcoin #Sportsbook   
498                                                                         Live Top 5 Crypto Prices &amp; 24hr Change (by Market Cap)\n\n1. #bitcoin  $ 34,011 -4.08%\n2. #ethereum  $ 2,238.68 -4.73%\n3. #tether  $ 1.00 0.00%\n4. #binance-coin  $ 303.08 -2.66%\n5. #cardano  $ 1.43 -2.05%\n\n#Doge #XRP #Crypto   
499                                                                                      Successful men and women keep moving forward; they make mistakes, but they don’t quit. Failure is only the opportunity to begin again, but t

In [15]:
tweets = tweets.set_index('Date').sort_index(ascending=False)

KeyError: "None of ['Date'] are in the columns"

In [16]:
tweets

text  \
Date                                                                                                                                                                                                                                                                                                                                     
2021-07-05 23:26:41  #Thailand #Boutiquehotels For Sale! \n#willroadhouse #AoyRoadhouse #compassgroupinternational #ThailandBeachfrontHomes #ThailandBeachfrontHome  #retirement \n#thailandrealestate #phuket #bkk\n#bangkok #huahin #jomtienbeach #jomtien #pattaya #thailandexpats #bitcoin #cryptocurrency https://t.co/GbFTpmaesz   
2021-07-05 23:26:36                            LUNALAND ($LLN) will burn coin supply until we reach 1 Lunar distance in coins. (400M coins) A burn rate of 99.96% from the initial pool of 1 Trillion coins reduces coin supply, #presale #blockchain #cryptotrading #investing #cryptocurrency #bitcoin #trading #altcoin #cryptonews   
2021-07-05 23:25:27                #LightningNetwork \n\nhttps://t.co/Q0CaYRhLWD observed:\n22,365 nodes (12,502 with active channels)\n53,035 channels\n1,682.728 BTC capacity ($57,335,747)\n\nmedian node capacity: 0.002 BTC ($74.64)\n\npast 24h:\n+72 nodes +972 channels +17.078 BTC ($581,892)\n\n#Lightning #LN #bitcoin $BTC   
2021-07-05 23:25:19                                    Barclay's just wants to keep you safe, this is why they let you spend your life savings on the lottery, at the casino, or trading stock futures. But buying #bitcoin  on Binance is a bridge to far. \n\nPS they are a huge criminal money launderer\n\nhttps://t.co/FUb9BMnavs   
2021-07-05 23:24:35                                                                                                                                                                                                                            Why 'Cardano is just teasing us' - https://t.co/8JowtgVKvj $ADA #altcoin #news #bitcoin   
...                                                                                                                                                                                                                                                                                                                                ...   
2021-07-05 20:24:17                                                                                        #linkedin #twitter #giftidea #facebook #instagram #tumblr #socialmedia #pinterest #dogecoin #reddit #gift #gifts #giftideas #shop #shopping #affiliatemarketing #bitcoin Shop #gifts &amp; #baskets https://t.co/dnN7uRmi6e   
2021-07-05 20:23:10                                                                                                                                   Argentina vs. Colombia: Copa America semifinals live stream, TV channel, how to watch online, news, odds https://t.co/ajfyDCgPlj ▶️ https://t.co/NKkDBVS66N #Bitcoin #Sportsbook   
2021-07-05 20:23:08                                                                                                                                           Brazil vs. Peru: Copa America semifinal live stream, TV channel, how to watch online, news, odds https://t.co/P86c6Ag9tV ▶️ https://t.co/NKkDBVS66N #Bitcoin #Sportsbook   
2021-07-05 20:22:53                                                                         Live Top 5 Crypto Prices &amp; 24hr Change (by Market Cap)\n\n1. #bitcoin  $ 34,011 -4.08%\n2. #ethereum  $ 2,238.68 -4.73%\n3. #tether  $ 1.00 0.00%\n4. #binance-coin  $ 303.08 -2.66%\n5. #cardano  $ 1.43 -2.05%\n\n#Doge #XRP #Crypto   
2021-07-05 20:22:40                                                                                      Successful men and women keep moving forward; they make mistakes, but they don’t quit. Failure is only the opportunity to begin again, but this time more intelligently. \n#MotivationMonday #Bitcoin https://t.co/P7G3ddjCfO   

                               

In [17]:
#Export to csv
output_file = 'Second_clean_twitter.csv'
tweets.to_csv(output_file, index='inplace')

In [18]:
# Sentiment labels function 
def sentiment_labels(df, feature, value): 
    df.loc[df[value] > 0,feature] = 'positive'
    df.loc[df[value] == 0,feature] = 'neutral'
    df.loc[df[value] < 0,feature] = 'negative'

In [19]:
# Blob analysis function 
def blob_sentiment(df):
    df['blob_polarity'] = df['final_clean'].apply(lambda x: TextBlob(x).sentiment.polarity)
    
    df['blob_subjectivity'] = df['final_clean'].apply(lambda x: TextBlob(x).sentiment.subjectivity)
    
    sentiment_labels(df, 'blob_sentiment', 'blob_polarity')
    
    return df[['final_clean', 'blob_polarity', 'blob_subjectivity', 'blob_sentiment']].head()
    

In [20]:
# Execute TextBlob Analysis and time
start = time.time()

blob_sentiment(tweets)

stop = time.time()

print(f'TextBlob analysis took: {round((stop-start)/60, 3)}minutes')

TextBlob analysis took: 0.03minutes


In [28]:
tweets.head(20)

text  \
Date                                                                                                                                                                                                                                                                                                                                     
2021-07-05 23:26:41  #Thailand #Boutiquehotels For Sale! \n#willroadhouse #AoyRoadhouse #compassgroupinternational #ThailandBeachfrontHomes #ThailandBeachfrontHome  #retirement \n#thailandrealestate #phuket #bkk\n#bangkok #huahin #jomtienbeach #jomtien #pattaya #thailandexpats #bitcoin #cryptocurrency https://t.co/GbFTpmaesz   
2021-07-05 23:26:36                            LUNALAND ($LLN) will burn coin supply until we reach 1 Lunar distance in coins. (400M coins) A burn rate of 99.96% from the initial pool of 1 Trillion coins reduces coin supply, #presale #blockchain #cryptotrading #investing #cryptocurrency #bitcoin #trading #altcoin #cryptonews   
2021-07-05 23:25:27                #LightningNetwork \n\nhttps://t.co/Q0CaYRhLWD observed:\n22,365 nodes (12,502 with active channels)\n53,035 channels\n1,682.728 BTC capacity ($57,335,747)\n\nmedian node capacity: 0.002 BTC ($74.64)\n\npast 24h:\n+72 nodes +972 channels +17.078 BTC ($581,892)\n\n#Lightning #LN #bitcoin $BTC   
2021-07-05 23:25:19                                    Barclay's just wants to keep you safe, this is why they let you spend your life savings on the lottery, at the casino, or trading stock futures. But buying #bitcoin  on Binance is a bridge to far. \n\nPS they are a huge criminal money launderer\n\nhttps://t.co/FUb9BMnavs   
2021-07-05 23:24:35                                                                                                                                                                                                                            Why 'Cardano is just teasing us' - https://t.co/8JowtgVKvj $ADA #altcoin #news #bitcoin   
2021-07-05 23:24:26                                                                                                                                                                                Ethereum Exchange Reserves Plummet To More Than Two-Year Low #Ethereum #bitcoin via https://t.co/gwG7Duur7Z https://t.co/eDz6P1tFqb   
2021-07-05 23:24:10                                                                                                                                                                                                                                                                   Who’s ready to win $100 of #Bitcoin right now 😎🤑   
2021-07-05 23:24:10                                                                                     #linkedin #twitter #giftsforhim #facebook #instagram #tumblr #socialmedia #pinterest #dogecoin #reddit #gift #gifts #giftideas #shop #shopping #affiliatemarketing #bitcoin Shop #gifts &amp; #baskets https://t.co/dnN7uRmi6e   
2021-07-05 23:24:09                                                                     #linkedin #twitter #giftsforhim #facebook #instagram #dogecoin #socialmedia #pinterest #doge #deal #gift #gifts #giftideas #shop #shopping #affiliatemarketing #bitcoin #freeshipping on purchases over $25 at #Amazon https://t.co/EynvCaoEnT   
2021-07-05 23:23:17                                                                                                                                                                             🔻🔻🔻 $LINA Short Position(v1) 🔻🔻🔻\nEntry Price: 0.03389\nTarget:0.03372\nStop:0.03423\n#Binance #Bitcoin #Signals #Sell #LINAUSDT #LINA   
2021-07-05 23:23:04                         Final drawn out sad ending to the MTI ponzi scheme. I remember so many people telling me I was an idiot for warning about this scam.These scammers are walking away with around 20,000 #bitcoin due to people wanting to believe something too good to be true.\n\nhttps://t.co/52KfMyty7u   
2021-07-05 23:22:10             

In [29]:
# Blob Analysis Df
blob_df = pd.DataFrame(tweets)
blob_df.drop(columns=['text', 'tokens', 'user_followers', 'user_friends', 'user_favourites','is_retweet', 'hashtags'], inplace=True)
blob_df

cleaned_text  \
Date                                                                                                                                                                                                                                                                                                            
2021-07-05 23:26:41   thailand  boutiquehotels for sale    willroadhouse  aoyroadhouse  compassgroupinternational  thailandbeachfronthomes  thailandbeachfronthome   retirement   thailandrealestate  phuket  bkk  bangkok  huahin  jomtienbeach  jomtien  pattaya  thailandexpats  bitcoin  cryptocurrency     
2021-07-05 23:26:36   lunaland   lln  will burn coin supply until we reach 1 lunar distance in coins   400m coins  a burn rate of 99 96  from the initial pool of 1 trillion coins reduces coin supply   presale  blockchain  cryptotrading  investing  cryptocurrency  bitcoin  trading  altcoin  cryptonews   
2021-07-05 23:25:27                          lightningnetwork     observed  22 365 nodes  12 502 with active channels  53 035 channels 1 682 728 btc capacity   57 335 747   median node capacity  0 002 btc   74 64   past 24h   72 nodes  972 channels  17 078 btc   581 892    lightning  ln  bitcoin  btc   
2021-07-05 23:25:19                                     barclay s just wants to keep you safe  this is why they let you spend your life savings on the lottery  at the casino  or trading stock futures  but buying  bitcoin  on binance is a bridge to far    ps they are a huge criminal money launderer      
2021-07-05 23:24:35                                                                                                                                                                                                                         why  cardano is just teasing us       ada  altcoin  news  bitcoin   
...                                                                                                                                                                                                                                                                                                       ...   
2021-07-05 20:24:17                                                                                      linkedin  twitter  giftidea  facebook  instagram  tumblr  socialmedia  pinterest  dogecoin  reddit  gift  gifts  giftideas  shop  shopping  affiliatemarketing  bitcoin shop  gifts  amp   baskets     
2021-07-05 20:23:10                                                                                                                                                      argentina vs  colombia  copa america semifinals live stream  tv channel  how to watch online  news  odds         bitcoin  sportsbook   
2021-07-05 20:23:08                                                                                                                                                              brazil vs  peru  copa america semifinal live stream  tv channel  how to watch online  news  odds         bitcoin  sportsbook   
2021-07-05 20:22:53                                                        live top 5 crypto prices  amp  24hr change  by market cap   1   bitcoin    34 011  4 08  2   ethereum    2 238 68  4 73  3   tether    1 00 0 00  4   binance coin    303 08  2 66  5   cardano    1 43  2 05    doge  xrp  crypto   
2021-07-05 20:22:40                                                                                    successful men and women keep moving forward  they make mistakes  but they don t quit  failure is only the opportunity to begin again  but this time more intelligently    motivationmonday  bitcoin     

                                                                                                                                                                                                                                                                     final_clean  \
Date                                           

In [45]:
# Vader sentiment analysis - define function and variable for SentimentIntensityAnalyzer()

sia = SentimentIntensityAnalyzer()


def vader_sentiment(df, clean=True):
    
    # Create two options to compare tokenized clean vs regex clean
    if clean:
        target_col='final_clean'
        prefix = 'vader_clean_'
    else:
        target_col='cleaned_text'
        prefix='vader_'
        
    scores_col=prefix+'scores'
    
    compound_col = prefix+'polarity'
    
    sentiment = prefix+'sentiment'
    
    df[scores_col] = df[target_col].apply(lambda x:sia.polarity_scores(x))
    
    df[compound_col] = df[scores_col].apply(lambda d: d['compound'])
    
    sentiment_labels(df, sentiment, compound_col)

In [46]:
#Execute vader function
start = time.time()

vader_sentiment(tweets, clean=True)

stop = time.time()

print(f'Vader analysis took: {round((stop-start)/60, 3)}minutes')

Vader analysis took: 0.002minutes


In [42]:
tweets.head(20)

text  \
Date                                                                                                                                                                                                                                                                                                                                     
2021-07-05 23:26:41  #Thailand #Boutiquehotels For Sale! \n#willroadhouse #AoyRoadhouse #compassgroupinternational #ThailandBeachfrontHomes #ThailandBeachfrontHome  #retirement \n#thailandrealestate #phuket #bkk\n#bangkok #huahin #jomtienbeach #jomtien #pattaya #thailandexpats #bitcoin #cryptocurrency https://t.co/GbFTpmaesz   
2021-07-05 23:26:36                            LUNALAND ($LLN) will burn coin supply until we reach 1 Lunar distance in coins. (400M coins) A burn rate of 99.96% from the initial pool of 1 Trillion coins reduces coin supply, #presale #blockchain #cryptotrading #investing #cryptocurrency #bitcoin #trading #altcoin #cryptonews   
2021-07-05 23:25:27                #LightningNetwork \n\nhttps://t.co/Q0CaYRhLWD observed:\n22,365 nodes (12,502 with active channels)\n53,035 channels\n1,682.728 BTC capacity ($57,335,747)\n\nmedian node capacity: 0.002 BTC ($74.64)\n\npast 24h:\n+72 nodes +972 channels +17.078 BTC ($581,892)\n\n#Lightning #LN #bitcoin $BTC   
2021-07-05 23:25:19                                    Barclay's just wants to keep you safe, this is why they let you spend your life savings on the lottery, at the casino, or trading stock futures. But buying #bitcoin  on Binance is a bridge to far. \n\nPS they are a huge criminal money launderer\n\nhttps://t.co/FUb9BMnavs   
2021-07-05 23:24:35                                                                                                                                                                                                                            Why 'Cardano is just teasing us' - https://t.co/8JowtgVKvj $ADA #altcoin #news #bitcoin   
2021-07-05 23:24:26                                                                                                                                                                                Ethereum Exchange Reserves Plummet To More Than Two-Year Low #Ethereum #bitcoin via https://t.co/gwG7Duur7Z https://t.co/eDz6P1tFqb   
2021-07-05 23:24:10                                                                                                                                                                                                                                                                   Who’s ready to win $100 of #Bitcoin right now 😎🤑   
2021-07-05 23:24:10                                                                                     #linkedin #twitter #giftsforhim #facebook #instagram #tumblr #socialmedia #pinterest #dogecoin #reddit #gift #gifts #giftideas #shop #shopping #affiliatemarketing #bitcoin Shop #gifts &amp; #baskets https://t.co/dnN7uRmi6e   
2021-07-05 23:24:09                                                                     #linkedin #twitter #giftsforhim #facebook #instagram #dogecoin #socialmedia #pinterest #doge #deal #gift #gifts #giftideas #shop #shopping #affiliatemarketing #bitcoin #freeshipping on purchases over $25 at #Amazon https://t.co/EynvCaoEnT   
2021-07-05 23:23:17                                                                                                                                                                             🔻🔻🔻 $LINA Short Position(v1) 🔻🔻🔻\nEntry Price: 0.03389\nTarget:0.03372\nStop:0.03423\n#Binance #Bitcoin #Signals #Sell #LINAUSDT #LINA   
2021-07-05 23:23:04                         Final drawn out sad ending to the MTI ponzi scheme. I remember so many people telling me I was an idiot for warning about this scam.These scammers are walking away with around 20,000 #bitcoin due to people wanting to believe something too good to be true.\n\nhttps://t.co/52KfMyty7u   
2021-07-05 23:22:10             

In [39]:
tweets.tail(20)

text  \
Date                                                                                                                                                                                                                                                               
2021-07-05 20:30:18  #linkedin #twitter #facebook #instagram #giftidea #dogecoin #socialmedia #pinterest #doge #deal #gift #gifts #giftideas #shop #shopping #affiliatemarketing #bitcoin #freeshipping on purchases over $25 at #Amazon https://t.co/EynvCaoEnT   
2021-07-05 20:30:18                                                                             BTC To form a GIANT Head &amp; Shoulders Pattern in #bitcoin.  for BITSTAMP:BTCUSD by INSANEMOANING #BTCUSD #btc https://t.co/RSwFXn9Aze https://t.co/P96sMh6LYU   
2021-07-05 20:30:04                                                                                                                Shoulder tear shelves Marlins prospect Sanchez https://t.co/6ySEA1odU4  ▶️ https://t.co/NKkDBVS66N #MLB #Bitcoin #Sportsbooks   
2021-07-05 20:28:55                                                                                                                                                                                                                #Bitcoin will reach to ______   
2021-07-05 20:28:38                                                                                                                                                                                                          #BITCOIN to 100k is inevitable EOY!   
2021-07-05 20:27:20                                                                                                                #bitcoin Is Ready to Fall Below 30,000 for BITSTAMP:BTCUSD by Dzinsai #BTCUSD https://t.co/4FYrfDcTKz https://t.co/Fp25EMS6xH   
2021-07-05 20:27:12  #linkedin #twitter #facebook #giftidea #instagram #dogecoin #socialmedia #pinterest #doge #deal #gift #gifts #giftideas #shop #shopping #affiliatemarketing #bitcoin #freeshipping on purchases over $25 at #Amazon https://t.co/EynvCaoEnT   
2021-07-05 20:27:11                  #linkedin #twitter #facebook #giftidea #instagram #tumblr #socialmedia #pinterest #dogecoin #reddit #gift #gifts #giftideas #shop #shopping #affiliatemarketing #bitcoin Shop #gifts &amp; #baskets https://t.co/dnN7uRmi6e   
2021-07-05 20:27:08                                                                                                                                                                                                             What happens first for #Bitcoin?   
2021-07-05 20:26:45                                                            Sound money is the purest language for communication. \n\nSound money regulates more than any government. \n\nSound money is the strongest clue between any society. \n\n#Bitcoin   
2021-07-05 20:26:12                                                                              @DocumentingBTC @FrancisSuarez Staking is always an option as well....#Bitcoin \n#BTC #cryptocurrency #Crypto #ETH #Binance #CryptoNews https://t.co/RHAZqY7Xhg   
2021-07-05 20:26:02                                                                                                               🚀🚀🚀 $BAT Long Position(v1) 🚀🚀🚀\nEntry Price: 0.5694\nTarget:0.5721\nStop:0.5635\n#Binance #Bitcoin #Signals #Buy #BATUSDT #BAT   
2021-07-05 20:25:51                                                                                                            🔻🔻🔻 $DGB Short Position(v2) 🔻🔻🔻\n\nEntry: 0.04352\nTarget:0.04331\nStop:0.04396\n\n#Binance #Bitcoin #Signals #Sell #DGBUSDT #DGB   
2021-07-05 20:25:28                                                     💵2,117 #BTC (74,964,500 USD) move from unknown wallet to unknown wallet\n \nDate : 2021-07-05 20:23:50 (GMT 0)\nBlockchain : #BITCOIN\nSymbol : #BTC\n \nDetail: https://t.co/P2P6fKBM0J   
2021-07-05 20:24:17  #linkedin #twitter #giftidea #facebook #instagram #dogecoin #socialmedi

In [47]:
# Execute vader function on cleaned_text instead of final_clean (compare scores)
vader_sentiment(tweets, clean=False)

In [49]:
tweets.head(20)

text  \
Date                                                                                                                                                                                                                                                                                                                                     
2021-07-05 23:26:41  #Thailand #Boutiquehotels For Sale! \n#willroadhouse #AoyRoadhouse #compassgroupinternational #ThailandBeachfrontHomes #ThailandBeachfrontHome  #retirement \n#thailandrealestate #phuket #bkk\n#bangkok #huahin #jomtienbeach #jomtien #pattaya #thailandexpats #bitcoin #cryptocurrency https://t.co/GbFTpmaesz   
2021-07-05 23:26:36                            LUNALAND ($LLN) will burn coin supply until we reach 1 Lunar distance in coins. (400M coins) A burn rate of 99.96% from the initial pool of 1 Trillion coins reduces coin supply, #presale #blockchain #cryptotrading #investing #cryptocurrency #bitcoin #trading #altcoin #cryptonews   
2021-07-05 23:25:27                #LightningNetwork \n\nhttps://t.co/Q0CaYRhLWD observed:\n22,365 nodes (12,502 with active channels)\n53,035 channels\n1,682.728 BTC capacity ($57,335,747)\n\nmedian node capacity: 0.002 BTC ($74.64)\n\npast 24h:\n+72 nodes +972 channels +17.078 BTC ($581,892)\n\n#Lightning #LN #bitcoin $BTC   
2021-07-05 23:25:19                                    Barclay's just wants to keep you safe, this is why they let you spend your life savings on the lottery, at the casino, or trading stock futures. But buying #bitcoin  on Binance is a bridge to far. \n\nPS they are a huge criminal money launderer\n\nhttps://t.co/FUb9BMnavs   
2021-07-05 23:24:35                                                                                                                                                                                                                            Why 'Cardano is just teasing us' - https://t.co/8JowtgVKvj $ADA #altcoin #news #bitcoin   
2021-07-05 23:24:26                                                                                                                                                                                Ethereum Exchange Reserves Plummet To More Than Two-Year Low #Ethereum #bitcoin via https://t.co/gwG7Duur7Z https://t.co/eDz6P1tFqb   
2021-07-05 23:24:10                                                                                                                                                                                                                                                                   Who’s ready to win $100 of #Bitcoin right now 😎🤑   
2021-07-05 23:24:10                                                                                     #linkedin #twitter #giftsforhim #facebook #instagram #tumblr #socialmedia #pinterest #dogecoin #reddit #gift #gifts #giftideas #shop #shopping #affiliatemarketing #bitcoin Shop #gifts &amp; #baskets https://t.co/dnN7uRmi6e   
2021-07-05 23:24:09                                                                     #linkedin #twitter #giftsforhim #facebook #instagram #dogecoin #socialmedia #pinterest #doge #deal #gift #gifts #giftideas #shop #shopping #affiliatemarketing #bitcoin #freeshipping on purchases over $25 at #Amazon https://t.co/EynvCaoEnT   
2021-07-05 23:23:17                                                                                                                                                                             🔻🔻🔻 $LINA Short Position(v1) 🔻🔻🔻\nEntry Price: 0.03389\nTarget:0.03372\nStop:0.03423\n#Binance #Bitcoin #Signals #Sell #LINAUSDT #LINA   
2021-07-05 23:23:04                         Final drawn out sad ending to the MTI ponzi scheme. I remember so many people telling me I was an idiot for warning about this scam.These scammers are walking away with around 20,000 #bitcoin due to people wanting to believe something too good to be true.\n\nhttps://t.co/52KfMyty7u   
2021-07-05 23:22:10             

In [55]:
# Calculate accuracy score of vader analysis on regex cleaned text vs tokenized text
from sklearn.metrics import accuracy_score

accuracy_score(tweets['vader_sentiment'], tweets['vader_clean_sentiment'])

0.956

In [57]:
# Vader df 
vader_columns = ['text', 'cleaned_text', 'tokens', 'final_clean', 'user_followers', 'user_friends', 'user_favourites','is_retweet', 'hashtags', 'vader_clean_scores', 'vader_clean_polarity', 'vader_clean_sentiment', 'vader_scores', 'vader_polarity', 'vader_sentiment']
vader_df = pd.DataFrame(tweets[vader_columns])
vader_df.head(20)

text  \
Date                                                                                                                                                                                                                                                                                                                                     
2021-07-05 23:26:41  #Thailand #Boutiquehotels For Sale! \n#willroadhouse #AoyRoadhouse #compassgroupinternational #ThailandBeachfrontHomes #ThailandBeachfrontHome  #retirement \n#thailandrealestate #phuket #bkk\n#bangkok #huahin #jomtienbeach #jomtien #pattaya #thailandexpats #bitcoin #cryptocurrency https://t.co/GbFTpmaesz   
2021-07-05 23:26:36                            LUNALAND ($LLN) will burn coin supply until we reach 1 Lunar distance in coins. (400M coins) A burn rate of 99.96% from the initial pool of 1 Trillion coins reduces coin supply, #presale #blockchain #cryptotrading #investing #cryptocurrency #bitcoin #trading #altcoin #cryptonews   
2021-07-05 23:25:27                #LightningNetwork \n\nhttps://t.co/Q0CaYRhLWD observed:\n22,365 nodes (12,502 with active channels)\n53,035 channels\n1,682.728 BTC capacity ($57,335,747)\n\nmedian node capacity: 0.002 BTC ($74.64)\n\npast 24h:\n+72 nodes +972 channels +17.078 BTC ($581,892)\n\n#Lightning #LN #bitcoin $BTC   
2021-07-05 23:25:19                                    Barclay's just wants to keep you safe, this is why they let you spend your life savings on the lottery, at the casino, or trading stock futures. But buying #bitcoin  on Binance is a bridge to far. \n\nPS they are a huge criminal money launderer\n\nhttps://t.co/FUb9BMnavs   
2021-07-05 23:24:35                                                                                                                                                                                                                            Why 'Cardano is just teasing us' - https://t.co/8JowtgVKvj $ADA #altcoin #news #bitcoin   
2021-07-05 23:24:26                                                                                                                                                                                Ethereum Exchange Reserves Plummet To More Than Two-Year Low #Ethereum #bitcoin via https://t.co/gwG7Duur7Z https://t.co/eDz6P1tFqb   
2021-07-05 23:24:10                                                                                                                                                                                                                                                                   Who’s ready to win $100 of #Bitcoin right now 😎🤑   
2021-07-05 23:24:10                                                                                     #linkedin #twitter #giftsforhim #facebook #instagram #tumblr #socialmedia #pinterest #dogecoin #reddit #gift #gifts #giftideas #shop #shopping #affiliatemarketing #bitcoin Shop #gifts &amp; #baskets https://t.co/dnN7uRmi6e   
2021-07-05 23:24:09                                                                     #linkedin #twitter #giftsforhim #facebook #instagram #dogecoin #socialmedia #pinterest #doge #deal #gift #gifts #giftideas #shop #shopping #affiliatemarketing #bitcoin #freeshipping on purchases over $25 at #Amazon https://t.co/EynvCaoEnT   
2021-07-05 23:23:17                                                                                                                                                                             🔻🔻🔻 $LINA Short Position(v1) 🔻🔻🔻\nEntry Price: 0.03389\nTarget:0.03372\nStop:0.03423\n#Binance #Bitcoin #Signals #Sell #LINAUSDT #LINA   
2021-07-05 23:23:04                         Final drawn out sad ending to the MTI ponzi scheme. I remember so many people telling me I was an idiot for warning about this scam.These scammers are walking away with around 20,000 #bitcoin due to people wanting to believe something too good to be true.\n\nhttps://t.co/52KfMyty7u   
2021-07-05 23:22:10             

In [78]:
#Flair model 

flair_classifier = TextClassifier.load('en-sentiment')
sentiment_model = flair.models.TextClassifier.load('en-sentiment')

2021-07-21 21:55:36,532 loading file /Users/memme11/.flair/models/sentiment-en-mix-distillbert_4.pt
2021-07-21 21:55:42,470 loading file /Users/memme11/.flair/models/sentiment-en-mix-distillbert_4.pt


In [61]:
#Flair functions 

# Function for splitting text into sentences
def flair_sentences(regex_text):
    sentences = [sentence for sentence in split_single(regex_text)]
    return sentences

# Function that creates numerical scores for sentiment judgements between -1 and 1
def flair_predict_sentences(sentence):
    if sentence == "":
        return 0 
    
    text=Sentence(sentence)
    
    flair_classifier.predict(text)
    value = text.labels[0].to_dict()['value']
    
    if value == 'POSITIVE':
        result = text.to_dict()['labels'][0]['confidence']
        
    else: 
        result = -(text.to_dict()['labels'][0]['confidence'])
        
    return round(result, 2)

# Function that executes the flair_predict_sentences function and adds it to the df
def flair_sentence_scores(sentences):
    results=[]
    
    for i in range(0, len(sentences)):
        results.append(flair_predict_sentences(sentences[i]))
        results.append(flair_predict_sentences(sentences[0]))
        
    return results

def flair_sum(scores):
    result = round(sum(scores), 2)
    return result

def avg_scores(scores):
    result = round(np.mean(scores), 2)
    return result

def flair_tweet_score(cleaned_text):
    if not cleaned_text:
        return 0
    
    tweet = Sentence(cleaned_text)
    flair_classifier.predict(tweet)
    value = tweet.labels[0].to_dict()['value']
    
    if value == 'POSITIVE':
        result = tweet.to_dict()['labels'][0]['confidence']
    
    else: 
        result = -(tweet.to_dict()['labels'][0]['confidence'])
        
    return round(result, 2)

def flair_sentiment(polarity):
    if polarity > 0:
        return 'positive'
    
    if polarity == 0:
        return 'neutral'
    
    elif polarity < 0:
        return 'negative'

In [72]:
# Flair per sentence 
# divide tweet into sentences, score them, then get avg scores of each sentence

start = time.time()

tweets['sentences'] = tweets['text'].apply(flair_sentences)
tweets['flair_scores'] = tweets['sentences'].apply(flair_sentence_scores)
tweets['flair_avg_score'] = tweets['flair_scores'].apply(avg_scores)

end = time.time()
print(round((stop-start)/60, 3))

-1631.98


In [73]:
tweets.head(20)

text  \
Date                                                                                                                                                                                                                                                                                                                                     
2021-07-05 23:26:41  #Thailand #Boutiquehotels For Sale! \n#willroadhouse #AoyRoadhouse #compassgroupinternational #ThailandBeachfrontHomes #ThailandBeachfrontHome  #retirement \n#thailandrealestate #phuket #bkk\n#bangkok #huahin #jomtienbeach #jomtien #pattaya #thailandexpats #bitcoin #cryptocurrency https://t.co/GbFTpmaesz   
2021-07-05 23:26:36                            LUNALAND ($LLN) will burn coin supply until we reach 1 Lunar distance in coins. (400M coins) A burn rate of 99.96% from the initial pool of 1 Trillion coins reduces coin supply, #presale #blockchain #cryptotrading #investing #cryptocurrency #bitcoin #trading #altcoin #cryptonews   
2021-07-05 23:25:27                #LightningNetwork \n\nhttps://t.co/Q0CaYRhLWD observed:\n22,365 nodes (12,502 with active channels)\n53,035 channels\n1,682.728 BTC capacity ($57,335,747)\n\nmedian node capacity: 0.002 BTC ($74.64)\n\npast 24h:\n+72 nodes +972 channels +17.078 BTC ($581,892)\n\n#Lightning #LN #bitcoin $BTC   
2021-07-05 23:25:19                                    Barclay's just wants to keep you safe, this is why they let you spend your life savings on the lottery, at the casino, or trading stock futures. But buying #bitcoin  on Binance is a bridge to far. \n\nPS they are a huge criminal money launderer\n\nhttps://t.co/FUb9BMnavs   
2021-07-05 23:24:35                                                                                                                                                                                                                            Why 'Cardano is just teasing us' - https://t.co/8JowtgVKvj $ADA #altcoin #news #bitcoin   
2021-07-05 23:24:26                                                                                                                                                                                Ethereum Exchange Reserves Plummet To More Than Two-Year Low #Ethereum #bitcoin via https://t.co/gwG7Duur7Z https://t.co/eDz6P1tFqb   
2021-07-05 23:24:10                                                                                                                                                                                                                                                                   Who’s ready to win $100 of #Bitcoin right now 😎🤑   
2021-07-05 23:24:10                                                                                     #linkedin #twitter #giftsforhim #facebook #instagram #tumblr #socialmedia #pinterest #dogecoin #reddit #gift #gifts #giftideas #shop #shopping #affiliatemarketing #bitcoin Shop #gifts &amp; #baskets https://t.co/dnN7uRmi6e   
2021-07-05 23:24:09                                                                     #linkedin #twitter #giftsforhim #facebook #instagram #dogecoin #socialmedia #pinterest #doge #deal #gift #gifts #giftideas #shop #shopping #affiliatemarketing #bitcoin #freeshipping on purchases over $25 at #Amazon https://t.co/EynvCaoEnT   
2021-07-05 23:23:17                                                                                                                                                                             🔻🔻🔻 $LINA Short Position(v1) 🔻🔻🔻\nEntry Price: 0.03389\nTarget:0.03372\nStop:0.03423\n#Binance #Bitcoin #Signals #Sell #LINAUSDT #LINA   
2021-07-05 23:23:04                         Final drawn out sad ending to the MTI ponzi scheme. I remember so many people telling me I was an idiot for warning about this scam.These scammers are walking away with around 20,000 #bitcoin due to people wanting to believe something too good to be true.\n\nhttps://t.co/52KfMyty7u   
2021-07-05 23:22:10             

In [74]:
# Flair per tweet
tweets['flair_cleaned_text'] = tweets['cleaned_text'].apply(flair_tweet_score)
tweets['flair_token'] = tweets['final_clean'].apply(flair_tweet_score)

In [76]:
tweets.head(30)

text  \
Date                                                                                                                                                                                                                                                                                                                                      
2021-07-05 23:26:41   #Thailand #Boutiquehotels For Sale! \n#willroadhouse #AoyRoadhouse #compassgroupinternational #ThailandBeachfrontHomes #ThailandBeachfrontHome  #retirement \n#thailandrealestate #phuket #bkk\n#bangkok #huahin #jomtienbeach #jomtien #pattaya #thailandexpats #bitcoin #cryptocurrency https://t.co/GbFTpmaesz   
2021-07-05 23:26:36                             LUNALAND ($LLN) will burn coin supply until we reach 1 Lunar distance in coins. (400M coins) A burn rate of 99.96% from the initial pool of 1 Trillion coins reduces coin supply, #presale #blockchain #cryptotrading #investing #cryptocurrency #bitcoin #trading #altcoin #cryptonews   
2021-07-05 23:25:27                 #LightningNetwork \n\nhttps://t.co/Q0CaYRhLWD observed:\n22,365 nodes (12,502 with active channels)\n53,035 channels\n1,682.728 BTC capacity ($57,335,747)\n\nmedian node capacity: 0.002 BTC ($74.64)\n\npast 24h:\n+72 nodes +972 channels +17.078 BTC ($581,892)\n\n#Lightning #LN #bitcoin $BTC   
2021-07-05 23:25:19                                     Barclay's just wants to keep you safe, this is why they let you spend your life savings on the lottery, at the casino, or trading stock futures. But buying #bitcoin  on Binance is a bridge to far. \n\nPS they are a huge criminal money launderer\n\nhttps://t.co/FUb9BMnavs   
2021-07-05 23:24:35                                                                                                                                                                                                                             Why 'Cardano is just teasing us' - https://t.co/8JowtgVKvj $ADA #altcoin #news #bitcoin   
2021-07-05 23:24:26                                                                                                                                                                                 Ethereum Exchange Reserves Plummet To More Than Two-Year Low #Ethereum #bitcoin via https://t.co/gwG7Duur7Z https://t.co/eDz6P1tFqb   
2021-07-05 23:24:10                                                                                                                                                                                                                                                                    Who’s ready to win $100 of #Bitcoin right now 😎🤑   
2021-07-05 23:24:10                                                                                      #linkedin #twitter #giftsforhim #facebook #instagram #tumblr #socialmedia #pinterest #dogecoin #reddit #gift #gifts #giftideas #shop #shopping #affiliatemarketing #bitcoin Shop #gifts &amp; #baskets https://t.co/dnN7uRmi6e   
2021-07-05 23:24:09                                                                      #linkedin #twitter #giftsforhim #facebook #instagram #dogecoin #socialmedia #pinterest #doge #deal #gift #gifts #giftideas #shop #shopping #affiliatemarketing #bitcoin #freeshipping on purchases over $25 at #Amazon https://t.co/EynvCaoEnT   
2021-07-05 23:23:17                                                                                                                                                                              🔻🔻🔻 $LINA Short Position(v1) 🔻🔻🔻\nEntry Price: 0.03389\nTarget:0.03372\nStop:0.03423\n#Binance #Bitcoin #Signals #Sell #LINAUSDT #LINA   
2021-07-05 23:23:04                          Final drawn out sad ending to the MTI ponzi scheme. I remember so many people telling me I was an idiot for warning about this scam.These scammers are walking away with around 20,000 #bitcoin due to people wanting to believe something too good to be true.\n\nhttps://t.co/52KfMyty7u   
2021-07-05 23:22:10 

In [85]:
tweets.tail(30)

text  \
Date                                                                                                                                                                                                                                                                                                 
2021-07-05 20:33:10                               #wedding #tumblr #twitter #facebook #instagram #dogecoin #socialmedia #pinterest #doge #deal #gift #gifts #giftideas #shop #shopping #affiliatemarketing #bitcoin Mrs. Fields Cookies - Shop #Cookies &amp; #GiftBaskets https://t.co/cxtRqfOUhP   
2021-07-05 20:33:09                                                    #linkedin #twitter #facebook #instagram #tumblr #giftidea #socialmedia #pinterest #dogecoin #reddit #gift #gifts #giftideas #shop #shopping #affiliatemarketing #bitcoin Shop #gifts &amp; #baskets https://t.co/dnN7uRmi6e   
2021-07-05 20:33:08                                                                                                                                                                                                       "Fascism, socialism, or #Bitcoin. Those are our choices." ― @tom_bibiyan   
2021-07-05 20:33:08                                    #linkedin #twitter #facebook #instagram #dogecoin #giftidea #socialmedia #pinterest #doge #deal #gift #gifts #giftideas #shop #shopping #affiliatemarketing #bitcoin #freeshipping on purchases over $25 at #Amazon https://t.co/EynvCaoEnT   
2021-07-05 20:32:43                                                                                                                                                                                                                                      Patience is still key here with #bitcoin.   
2021-07-05 20:32:01                                             When you're trying to reinvent traditional payment systems and challenge the banking hegemony directly, you have to expect attacks from all sides. The path to freedom is not easy, but it's made a whole lot easier with #Bitcoin   
2021-07-05 20:31:58                                                                                                                                                               📉💴$1,657,960 #bitcoin SHORTED @$33999.98 [05/07/21 20:31:57]\n🏭 bybit | $BTCUSD\n\n🗣️Hodl! Lambo soon! - 33_kobo   
2021-07-05 20:31:20  New airdrop: Icarus Network (ICA)\nTotal Reward: 150,000 ICA\nRate: ⭐️⭐️⭐️⭐️\nWinners: 300 Randomly selected participants\nDistribution: 14 days after the first listing\n\nAirdrop details: https://t.co/nXVJJRD9eL\n\n#Airdrop #Airdrops #Airdropinspector #Crypto #Bitcoin   
2021-07-05 20:31:03                                                                                                                                               🔻🔻🔻 $ZRX Short Position(v1) 🔻🔻🔻\nEntry Price: 0.7011\nTarget:0.6977\nStop:0.7082\n#Binance #Bitcoin #Signals #Sell #ZRXUSDT #ZRX   
2021-07-05 20:30:19                                                    #linkedin #twitter #facebook #instagram #giftidea #tumblr #socialmedia #pinterest #dogecoin #reddit #gift #gifts #giftideas #shop #shopping #affiliatemarketing #bitcoin Shop #gifts &amp; #baskets https://t.co/dnN7uRmi6e   
2021-07-05 20:30:18                                    #linkedin #twitter #facebook #instagram #giftidea #dogecoin #socialmedia #pinterest #doge #deal #gift #gifts #giftideas #shop #shopping #affiliatemarketing #bitcoin #freeshipping on purchases over $25 at #Amazon https://t.co/EynvCaoEnT   
2021-07-05 20:30:18                                                                                                               BTC To form a GIANT Head &amp; Shoulders Pattern in #bitcoin.  for BITSTAMP:BTCUSD by INSANEMOANING #BTCUSD #btc https://t.co/RSwFXn9Aze https://t.co/P96sMh6LYU   
2021-07-05 20:30:04                                                                                                                                                  Shoul

In [ ]:
# Sentiment labels for each type of flair analysis
probs = []
sentiment = []

for tweet in tweets['final_clean'].to_list():
    
    # make prediction
    sentence = flair.data.Sentence(tweet)
    sentiment_model.predict(sentence)
    
    # extract sentiment prediction
    
    sentiment.append(sentence.labels[0].value)  # numerical score 0-1
    probs.append(sentence.labels[0].value)

# add probability and sentiment predictions to tweets dataframe
tweets['sentiment'] = sentiment

In [92]:
tweets['uncleaned_sentiment'] = tweets['flair_cleaned_text'].apply(flair_sentiment)
tweets['avg_sentiment'] = tweets['flair_avg_score'].apply(flair_sentiment)

In [86]:
flair_columns = ['text', 'cleaned_text', 'final_clean', 'user_followers', 'user_friends', 'user_favourites','is_retweet', 'hashtags', 'flair_avg_score', 'flair_cleaned_text', 'flair_token', 'probability1', 'sentiment']

flair_df = pd.DataFrame(tweets[flair_columns])

In [93]:
flair_df['uncleaned_sentiment'] = tweets['uncleaned_sentiment']
flair_df['avg_sentiment'] = tweets['avg_sentiment']

In [94]:
flair_df

text  \
Date                                                                                                                                                                                                                                                                                                                                     
2021-07-05 23:26:41  #Thailand #Boutiquehotels For Sale! \n#willroadhouse #AoyRoadhouse #compassgroupinternational #ThailandBeachfrontHomes #ThailandBeachfrontHome  #retirement \n#thailandrealestate #phuket #bkk\n#bangkok #huahin #jomtienbeach #jomtien #pattaya #thailandexpats #bitcoin #cryptocurrency https://t.co/GbFTpmaesz   
2021-07-05 23:26:36                            LUNALAND ($LLN) will burn coin supply until we reach 1 Lunar distance in coins. (400M coins) A burn rate of 99.96% from the initial pool of 1 Trillion coins reduces coin supply, #presale #blockchain #cryptotrading #investing #cryptocurrency #bitcoin #trading #altcoin #cryptonews   
2021-07-05 23:25:27                #LightningNetwork \n\nhttps://t.co/Q0CaYRhLWD observed:\n22,365 nodes (12,502 with active channels)\n53,035 channels\n1,682.728 BTC capacity ($57,335,747)\n\nmedian node capacity: 0.002 BTC ($74.64)\n\npast 24h:\n+72 nodes +972 channels +17.078 BTC ($581,892)\n\n#Lightning #LN #bitcoin $BTC   
2021-07-05 23:25:19                                    Barclay's just wants to keep you safe, this is why they let you spend your life savings on the lottery, at the casino, or trading stock futures. But buying #bitcoin  on Binance is a bridge to far. \n\nPS they are a huge criminal money launderer\n\nhttps://t.co/FUb9BMnavs   
2021-07-05 23:24:35                                                                                                                                                                                                                            Why 'Cardano is just teasing us' - https://t.co/8JowtgVKvj $ADA #altcoin #news #bitcoin   
...                                                                                                                                                                                                                                                                                                                                ...   
2021-07-05 20:24:17                                                                                        #linkedin #twitter #giftidea #facebook #instagram #tumblr #socialmedia #pinterest #dogecoin #reddit #gift #gifts #giftideas #shop #shopping #affiliatemarketing #bitcoin Shop #gifts &amp; #baskets https://t.co/dnN7uRmi6e   
2021-07-05 20:23:10                                                                                                                                   Argentina vs. Colombia: Copa America semifinals live stream, TV channel, how to watch online, news, odds https://t.co/ajfyDCgPlj ▶️ https://t.co/NKkDBVS66N #Bitcoin #Sportsbook   
2021-07-05 20:23:08                                                                                                                                           Brazil vs. Peru: Copa America semifinal live stream, TV channel, how to watch online, news, odds https://t.co/P86c6Ag9tV ▶️ https://t.co/NKkDBVS66N #Bitcoin #Sportsbook   
2021-07-05 20:22:53                                                                         Live Top 5 Crypto Prices &amp; 24hr Change (by Market Cap)\n\n1. #bitcoin  $ 34,011 -4.08%\n2. #ethereum  $ 2,238.68 -4.73%\n3. #tether  $ 1.00 0.00%\n4. #binance-coin  $ 303.08 -2.66%\n5. #cardano  $ 1.43 -2.05%\n\n#Doge #XRP #Crypto   
2021-07-05 20:22:40                                                                                      Successful men and women keep moving forward; they make mistakes, but they don’t quit. Failure is only the opportunity to begin again, but this time more intelligently. \n#MotivationMonday #Bitcoin https://t.co/P7G3ddjCfO   

                               

In [130]:
vader_df

text  \
Date                                                                                                                                                                                                                                                                                                                                     
2021-07-05 23:26:41  #Thailand #Boutiquehotels For Sale! \n#willroadhouse #AoyRoadhouse #compassgroupinternational #ThailandBeachfrontHomes #ThailandBeachfrontHome  #retirement \n#thailandrealestate #phuket #bkk\n#bangkok #huahin #jomtienbeach #jomtien #pattaya #thailandexpats #bitcoin #cryptocurrency https://t.co/GbFTpmaesz   
2021-07-05 23:26:36                            LUNALAND ($LLN) will burn coin supply until we reach 1 Lunar distance in coins. (400M coins) A burn rate of 99.96% from the initial pool of 1 Trillion coins reduces coin supply, #presale #blockchain #cryptotrading #investing #cryptocurrency #bitcoin #trading #altcoin #cryptonews   
2021-07-05 23:25:27                #LightningNetwork \n\nhttps://t.co/Q0CaYRhLWD observed:\n22,365 nodes (12,502 with active channels)\n53,035 channels\n1,682.728 BTC capacity ($57,335,747)\n\nmedian node capacity: 0.002 BTC ($74.64)\n\npast 24h:\n+72 nodes +972 channels +17.078 BTC ($581,892)\n\n#Lightning #LN #bitcoin $BTC   
2021-07-05 23:25:19                                    Barclay's just wants to keep you safe, this is why they let you spend your life savings on the lottery, at the casino, or trading stock futures. But buying #bitcoin  on Binance is a bridge to far. \n\nPS they are a huge criminal money launderer\n\nhttps://t.co/FUb9BMnavs   
2021-07-05 23:24:35                                                                                                                                                                                                                            Why 'Cardano is just teasing us' - https://t.co/8JowtgVKvj $ADA #altcoin #news #bitcoin   
...                                                                                                                                                                                                                                                                                                                                ...   
2021-07-05 20:24:17                                                                                        #linkedin #twitter #giftidea #facebook #instagram #tumblr #socialmedia #pinterest #dogecoin #reddit #gift #gifts #giftideas #shop #shopping #affiliatemarketing #bitcoin Shop #gifts &amp; #baskets https://t.co/dnN7uRmi6e   
2021-07-05 20:23:10                                                                                                                                   Argentina vs. Colombia: Copa America semifinals live stream, TV channel, how to watch online, news, odds https://t.co/ajfyDCgPlj ▶️ https://t.co/NKkDBVS66N #Bitcoin #Sportsbook   
2021-07-05 20:23:08                                                                                                                                           Brazil vs. Peru: Copa America semifinal live stream, TV channel, how to watch online, news, odds https://t.co/P86c6Ag9tV ▶️ https://t.co/NKkDBVS66N #Bitcoin #Sportsbook   
2021-07-05 20:22:53                                                                         Live Top 5 Crypto Prices &amp; 24hr Change (by Market Cap)\n\n1. #bitcoin  $ 34,011 -4.08%\n2. #ethereum  $ 2,238.68 -4.73%\n3. #tether  $ 1.00 0.00%\n4. #binance-coin  $ 303.08 -2.66%\n5. #cardano  $ 1.43 -2.05%\n\n#Doge #XRP #Crypto   
2021-07-05 20:22:40                                                                                      Successful men and women keep moving forward; they make mistakes, but they don’t quit. Failure is only the opportunity to begin again, but this time more intelligently. \n#MotivationMonday #Bitcoin https://t.co/P7G3ddjCfO   

                               

In [90]:
blob_df['blob_sentiment'].value_counts()

neutral     313
positive    126
negative     61
Name: blob_sentiment, dtype: int64

In [89]:
vader_df['vader_sentiment'].value_counts()

positive    297
neutral     104
negative     99
Name: vader_sentiment, dtype: int64

In [91]:
flair_df['sentiment'].value_counts()

POSITIVE    251
NEGATIVE    249
Name: sentiment, dtype: int64

In [95]:
flair_df['uncleaned_sentiment'].value_counts()

positive    267
negative    233
Name: uncleaned_sentiment, dtype: int64

In [96]:
flair_df['avg_sentiment'].value_counts()

positive    272
negative    228
Name: avg_sentiment, dtype: int64

In [100]:
flair_values = flair_df.loc[:, 'flair_token']

Date
2021-07-05 23:26:41    1.00
2021-07-05 23:26:36   -0.85
2021-07-05 23:25:27   -0.79
2021-07-05 23:25:19   -1.00
2021-07-05 23:24:35   -1.00
                       ... 
2021-07-05 20:24:17    0.92
2021-07-05 20:23:10    0.69
2021-07-05 20:23:08    0.73
2021-07-05 20:22:53   -0.57
2021-07-05 20:22:40   -0.84
Name: flair_token, Length: 500, dtype: float64

In [138]:
def pos_neg_neutral_avg(df):
    
    positive = []
    neutral = []
    negative = []
    
    for values in df:
        if values > 0:
            positive.append(values)
        
        elif values < 0:
            negative.append(values)
        
        else:
            neutral.append(values)
    
    print(f'Positive score average for {df.name} = {round(np.mean(positive), 2)}')
    print(f'Neutral score average for {df.name} = {round(np.mean(neutral), 2)}')
    print(f'Positive score average for {df.name} = {round(np.mean(negative), 2)}')

In [139]:
textblob_values = blob_df.loc[:, 'blob_polarity']
vader_values = vader_df.loc[:, 'vader_polarity']
flair_values = flair_df.loc[:, 'flair_token']

In [140]:
pos_neg_neutral_avg(textblob_values)

Positive score average for blob_polarity = 0.32
Neutral score average for blob_polarity = 0.0
Positive score average for blob_polarity = -0.15


In [151]:
round(np.mean(textblob_values), 4)

0.0623

In [141]:
pos_neg_neutral_avg(vader_values)

Positive score average for vader_polarity = 0.52
Neutral score average for vader_polarity = 0.0
Positive score average for vader_polarity = -0.39


In [150]:
round(np.mean(vader_values), 4)

0.2309

In [142]:
pos_neg_neutral_avg(flair_values)

Positive score average for flair_token = 0.92
Neutral score average for flair_token = nan
Positive score average for flair_token = -0.93


/Users/memme11/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3373: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/Users/memme11/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [149]:
round(np.mean(flair_values), 4)

-0.0006